In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras


import matplotlib.pyplot as plt
        

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


In [ ]:
train_data = pd.read_csv('../input/train.csv').values.astype('float32')
train_images = train_data[:,1:]
train_labels = train_data[:,0].astype('int32')

test_images = pd.read_csv('../input/test.csv').values.astype('float32')

In [ ]:
from sklearn.model_selection import train_test_split

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, train_size=0.8, random_state=100)

In [ ]:
print(np.shape(train_images))
print(np.shape(val_images))
print(np.shape(train_labels))
print(np.shape(val_labels))
print(np.shape(test_images))

In [ ]:
sample_images = train_images.reshape(train_images.shape[0], 28,28)
for i in range(1,6):
    plt.subplot(1,6,i)
    plt.axis('off')
    plt.set_cmap('gray')
    plt.title(train_labels[i])
    plt.imshow(sample_images[i])
    

In [ ]:
from keras.utils.np_utils import to_categorical
train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
        rescale=1./255,
        zca_whitening=False,
        rotation_range=10.,
               
)

val_data_generator = ImageDataGenerator(
        rescale=1./255     
)

In [ ]:
np.random.seed(100)

In [ ]:
from keras.models import Sequential

model = Sequential()

In [ ]:
from keras.layers import Dense, Flatten, Activation
from keras.layers.pooling import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers

#Reshapa data from, 2-Dimension into 4-Dimension
train_images = train_images.reshape(-1,28,28,1)
val_images = val_images.reshape(-1,28,28,1)

print(train_images.shape)

model.add(Conv2D(filters=32, kernel_size=[3,3], input_shape=(28,28,1),
                 padding="SAME", kernel_initializer='he_normal', 
                 kernel_regularizer=regularizers.l2(1e-8)))
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
print(model.output_shape)



model.add( Conv2D( filters=80, kernel_size=[3,3], padding="SAME", kernel_initializer='he_normal', 
                 kernel_regularizer=regularizers.l2(1e-8)
                 ))          
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
print(model.output_shape)

model.add( Conv2D( filters=32, kernel_size=[3,3], padding="SAME", kernel_initializer='he_normal', 
                 kernel_regularizer=regularizers.l2(1e-8)))          
model.add(BatchNormalization())
model.add(Activation(activation='relu'))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
print(model.output_shape)



model.add(Flatten())
print(model.output_shape)
     
          
model.add( Dense(units=100, activation='relu', kernel_initializer='he_normal'))
print(model.output_shape)         

          
model.add( Dense(units=10, activation='softmax', kernel_initializer='he_normal'))
print(model.output_shape)

In [ ]:
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping

adam = optimizers.Adamax(lr=0.001, decay=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam,
              metrics=['accuracy'])

Check=[EarlyStopping(monitor='val_acc', min_delta=0.00001, patience=2, verbose=1, mode='auto'), 
       ModelCheckpoint(filepath="/tmp/model.hdf5" , monitor='val_acc', verbose=2, save_best_only=True, mode='max')]

In [ ]:
data_generator.fit(train_images)

BATCH_SIZE = 25


hist = model.fit_generator(data_generator.flow(train_images, train_labels, batch_size=BATCH_SIZE),
             validation_data=val_data_generator.flow(val_images, val_labels, batch_size=BATCH_SIZE),
             steps_per_epoch=len(train_images)/(BATCH_SIZE), validation_steps=len(val_images)/(BATCH_SIZE), epochs=5,callbacks=Check, verbose=2)

In [ ]:
from keras.models import load_model

saved_model =  load_model('/tmp/model.hdf5')


In [ ]:
print(hist.history.keys())
plt.plot(hist.history['loss'])

In [ ]:
test_images = test_images/255
predictions = model.predict_classes(test_images.reshape(test_images.shape[0], 28, 28, 1), verbose=1)

In [ ]:
print("Predicted Samples are : ", predictions[:10])

sample_test_images = test_images.reshape(test_images.shape[0], 28,28)
      
for i in range(10):
    plt.subplot(1, 11, i+1)
    plt.axis('off')
    plt.set_cmap('gray')
    plt.title(predictions[i])
    plt.imshow(sample_test_images[i])

In [ ]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("submission.csv", index=False, header=True)

In [ ]:
submissions.head()